In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle
from empiricaldist import Cdf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model, svm
from sklearn.ensemble import StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, TweedieRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from yellowbrick.regressor import ResidualsPlot
import pygad
import random
import math
import matplotlib.pyplot as plt
import sql 

In [ ]:
with open('../Modelos/Comp_a/RandomForestRegressor_modelo.pkl', 'rb') as archivo:
    modelo_a = pickle.load(archivo)
with open('../Modelos/Comp_b/RandomForestRegressor_modelo.pkl', 'rb') as archivo:
    modelo_b = pickle.load(archivo)
with open('../Modelos/Comp_c/RandomForestRegressor_modelo.pkl', 'rb') as archivo:
    modelo_c = pickle.load(archivo)
with open('../Modelos/Comp_d/RandomForestRegressor_modelo.pkl', 'rb') as archivo:
    modelo_d = pickle.load(archivo)


with open('../Modelos/Comp_a/scaler_model.pkl', 'rb') as archivo:
    scaler_a = pickle.load(archivo)
with open('../Modelos/Comp_b/scaler_model.pkl', 'rb') as archivo:
    scaler_b = pickle.load(archivo)
with open('../Modelos/Comp_c/scaler_model.pkl', 'rb') as archivo:
    scaler_c = pickle.load(archivo)
with open('../Modelos/Comp_d/scaler_model.pkl', 'rb') as archivo:
    scaler_d = pickle.load(archivo)

condiciones_iniciales = [25, 25, 25, 25]

# Generamos la poblacion inicial
random.seed(42) #semilla para que en todas las ejecuciones la poblacion inicial sea la misma
poblacion_inicial = []
poblacion_inicial.append(condiciones_iniciales)
for _ in range(50):
    individuo = []
    for _ in range(4):
        gen = random.uniform(1, 90)
        individuo.append(gen)
    poblacion_inicial.append(individuo)


    #definimos la funcion fitness
def fitness_func(ga_instance, solution, solution_idx):
    capacidad = np.array([100, 90, 95, 110])
    frecuencia_porcentaje = [numero / 100 for numero in solution]
    volumen_aire  = capacidad * frecuencia_porcentaje
    if sum(volumen_aire) >= objetivo:
        comp_a = pd.DataFrame({
        'Presion': [presion_comp_a],
        'Temperatura': [temp_comp_a],
        'Frecuencia': solution[0]
        })
        X = scaler_d.transform(comp_a)
        X = pd.DataFrame(X, columns=comp_a.columns)
        potencia_a = modelo_a.predict(X)
        comp_b = pd.DataFrame({
        'Presion': [presion_comp_b],
        'Temperatura': [temp_comp_b],
        'Frecuencia': solution[1]
        })
        X = scaler_d.transform(comp_b)
        X = pd.DataFrame(X, columns=comp_b.columns)
        potencia_b = modelo_b.predict(X)
        comp_c = pd.DataFrame({
        'Presion': [presion_comp_c],
        'Temperatura': [temp_comp_c],
        'Frecuencia': solution[2]
        })
        X = scaler_d.transform(comp_c)
        X = pd.DataFrame(X, columns=comp_c.columns)
        potencia_c = modelo_c.predict(X)
        comp_d = pd.DataFrame({
        'Presion': [presion_comp_d],
        'Temperatura': [temp_comp_d],
        'Frecuencia': solution[3]
        })
        X = scaler_d.transform(comp_d)
        X = pd.DataFrame(X, columns=comp_d.columns)
        potencia_d = modelo_d.predict(X)
        fitness = (potencia_a + potencia_b + potencia_c + potencia_d)
        penalizacion = sum(volumen_aire) - objetivo
        fitness = (fitness)+math.exp(penalizacion)
        for i in range(len(solution)):
            if solution[i] != condiciones_iniciales[i]:
                fitness +=3
        fitness = -fitness
        fitness = float(fitness[0])
    else:
        fitness = -10e99
    return fitness


num_generations = 100000
num_parents_mating = len(poblacion_inicial)//2
num_genes = len(poblacion_inicial[0])
mutation_percent_genes = 1
mutation_probability = 0.1
keep_parents = 2
gene_space = range(1, 90)
parent_1 = 'ramdom'
crossover_1 = 'two_points'
mutation_1 = 'swap'


def on_stop(ga_instance, last_population_fitness):
    print(f"El algoritmo genético ha finalizado.")
    print("Mejor solución encontrada:", ga_instance.best_solution())

def on_generation(ga_instance):
    print("Generación {generation}: Mejor solucion = {best_solution} Mejor fitness = {best_fitness}".format(generation=ga_instance.generations_completed,
                                                                          best_solution = ga_instance.best_solution()[0],
                                                                          best_fitness = ga_instance.best_solution()[1]))

In [ ]:
presion = 0.75
temperatura = 14
objetivo = 200

In [ ]:
presion_comp_a = presion
temp_comp_a = temperatura
presion_comp_b = presion
temp_comp_b = temperatura
presion_comp_c = presion
temp_comp_c = temperatura
presion_comp_d = presion
temp_comp_d = temperatura
objetivo = aire
ga_instance_1 = pygad.GA(fitness_func=fitness_func,
                       num_generations=100000, 
                       num_parents_mating=num_parents_mating,
                       num_genes=num_genes,
                       parent_selection_type=parent_1,
                       crossover_type=crossover_1,
                       mutation_type= mutation_1,
                       gene_space=gene_space,
                       mutation_percent_genes = mutation_percent_genes,
                       mutation_probability=mutation_probability,
                       initial_population=poblacion_inicial,
                       gene_type=float,
                       on_stop=on_stop,
                       on_generation=on_generation,
                       save_solutions=True,
                       stop_criteria="saturate_25")
ga_instance_1.run()
comp_a = ga_instance_1.best_solution()[0][0]
comp_b = ga_instance_1.best_solution()[0][1]
comp_c = ga_instance_1.best_solution()[0][2]
comp_d = ga_instance_1.best_solution()[0][3]